In [ ]:
import pandas as pd
from setups_paper_gwp import *

path_base = Path('/data/user/kim_a/')
num_params = 35000
num_influential = 60
if num_params == 10000:
    iter_corr = 4*num_params
elif num_params == 35000:
    iter_corr = 2*num_params
gsa_corr = setup_corr(num_params, iter_corr, setup_lca_model_oases, path_base)
S_dict = gsa_corr.perform_gsa()
spearman = S_dict['spearman']
lca = gsa_corr.model.lca
where_inf = np.argsort(spearman)[::-1][:num_influential]
imp_tech = gsa_corr.model.uncertain_tech_params[where_inf]

In [ ]:
%%time
row_acts, col_acts = [],[]
for p in imp_tech:
    row = p['row']
    col = p['col']
    row_acts.append(bw.get_activity(lca.reverse_dict()[0][row]))
    col_acts.append(bw.get_activity(lca.reverse_dict()[0][col]))

In [ ]:
spearman_inf = spearman[where_inf]
data = []
for i in range(num_influential):
    row_act = row_acts[i]
    col_act = row_acts[i]
    dict_ = {
        "input activity": row_act.get("name"), 
        "input location": row_act.get("location"),
        "input reference product": row_act.get("reference product"),
        "input unit": row_act.get('unit'),

        "output activity": col_act.get("name"), 
        "output location": col_act.get("location"),
        "output reference product": col_act.get("reference product"),
        "output unit": col_act.get('unit'),
        
        "exchange amount": imp_tech["amount"][i],
        "exchange scale": imp_tech["scale"][i],
        "exchange loc": imp_tech["loc"][i],
        "exchange Spearman rank coef.": spearman_inf[i]
    }
    data.append(dict_)
df = pd.DataFrame.from_dict(data)
df.to_excel("influential_exchanges.xlsx")